<a href="https://colab.research.google.com/github/jinudaniel/AV-LTFS-Finhack2/blob/master/roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Oct 15 09:08:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import re
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
tqdm.pandas()

In [ ]:
train = pd.read_json("/content/drive/My Drive/Machine Hack/GitHub Bugs Prediction/data/embold_train.json").reset_index(drop=True)
test = pd.read_json("/content/drive/My Drive/Machine Hack/GitHub Bugs Prediction/data/embold_test.json").reset_index(drop=True)
train_extra = pd.read_json("/content/drive/My Drive/Machine Hack/GitHub Bugs Prediction/data/embold_train_extra.json").reset_index(drop=True)

In [ ]:
print(train.shape)
train = pd.concat([train, train_extra], ignore_index=True)
print(train.shape)

(150000, 3)
(450000, 3)


In [ ]:
# train = train.iloc[:1000, :]
# test = test.iloc[:200, :]
# train.shape, test.shape

((1000, 3), (200, 2))

In [ ]:
!pip install textsearch
!pip install contractions

     |████████████████████████████████| 317kB 7.1MB/s 
     |████████████████████████████████| 245kB 17.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81703 sha256=921edad2e67b4a0a9b4510418133589d00d294d491bfc72db0a963df338a4c66
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
import nltk
import re
from bs4 import BeautifulSoup
import unicodedata
import contractions
import spacy
import nltk
import tqdm
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nlp = spacy.load('en', parse=False, tag=False, entity=False)
ps = nltk.porter.PorterStemmer()


def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text


def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


def expand_contractions(text):
    return contractions.fix(text)


def spacy_lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text


def simple_stemming(text, stemmer=ps):
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_stopwords(text, is_lower_case=False, stopwords=None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    
    filtered_text = ' '.join(filtered_tokens) 
    return filtered_text

def remove_common_word(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    common_words = ['hey', 'hello', 'hi']
    filtered_tokens = [token for token in tokens if token.lower() not in common_words]
    filtered_text = ' '.join(filtered_tokens) 
    return filtered_text

def text_pre_processor(text, html_strip=True, accented_char_removal=True, contraction_expansion=True,
                       text_lower_case=True, special_char_removal=True, remove_digits=True, stopword_removal=False, 
                       stopword_list=None, text_stemming=False, text_lemmatization=False, remove_common_words=True):
    
    #remove urls
    text = re.sub(r'http\S+', '', text)
    
    # strip HTML
    if html_strip:
        text = strip_html_tags(text)
    
    # remove extra newlines (often might be present in really noisy text)
    text = text.translate(text.maketrans("\n\t\r", "   "))
    
    
    # remove accented characters
    if accented_char_removal:
        text = remove_accented_chars(text)
    
    # expand contractions    
    if contraction_expansion:
        text = expand_contractions(text)
        
        
    # remove special characters and\or digits    
    if special_char_removal:
        # insert spaces between special characters to isolate them    
        special_char_pattern = re.compile(r'([{.(-)!}])')
        text = special_char_pattern.sub(" \\1 ", text)
        text = remove_special_characters(text, remove_digits=remove_digits)
        
         
    # lowercase the text    
    if text_lower_case:
        text = text.lower()
        
        
    # remove stopwords
    if stopword_removal:
        text = remove_stopwords(text, is_lower_case=text_lower_case, 
                                stopwords=stopword_list)
    if remove_common_words:
      text = remove_common_word(text)
        
    # remove extra whitespace
    text = re.sub(' +', ' ', text)
    text = text.strip()
    
    return text

In [ ]:
train['clean_title'] = train['title'].progress_apply(text_pre_processor)
train['clean_body'] = train['body'].progress_apply(text_pre_processor)

100%|██████████| 450000/450000 [05:23<00:00, 1390.22it/s]


In [ ]:
test['clean_body'] = test['body'].progress_apply(text_pre_processor)
test['clean_title'] = test['title'].progress_apply(text_pre_processor)

100%|██████████| 30000/30000 [00:07<00:00, 3848.96it/s]


In [ ]:
train.loc[50000, ["clean_body", "body"]]

clean_body    test pyare pythonr a map lambda x x are r when...
body          test.py:\r    python\r a = map lambda x: x + 1...
Name: 50000, dtype: object

In [ ]:
train['text'] = train['clean_title'] + ' ' + train['clean_body']
test['text'] = test['clean_title'] + ' ' + test['clean_body']

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.1MB 4.9MB/s 
     |████████████████████████████████| 3.0MB 28.0MB/s 
     |████████████████████████████████| 1.1MB 46.1MB/s 
     |████████████████████████████████| 890kB 50.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c8721b508f7f22890f1d60dab11742ee40599dcf73cbe0072e7c242bbb80183f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import transformers
from transformers import RobertaModel, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
import random
import os
SEED = 424
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(SEED)

In [ ]:
PRE_TRAINED_MODEL_NAME = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
df_train, df_valid = train_test_split(train, test_size=0.1, random_state=SEED, 
                                      stratify=train['label'].values)
df_train.shape, df_valid.shape, test.shape

((405000, 6), (45000, 6), (30000, 5))

In [ ]:
MAX_LEN = 100

class GHClassificationDataset(Dataset):

  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.reviews)
  
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]

    encoding = self.tokenizer(review, add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding='max_length',
      return_attention_mask=True,
      truncation=True,
      return_tensors='pt')

    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GHClassificationDataset(
    reviews=df.text.to_numpy(),
    targets= df.label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [ ]:
test['label'] = 0

In [ ]:
BATCH_SIZE = 32

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_valid, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
import torch.nn.functional as F
hidden_dim = 256
class Classifier(nn.Module):

  def __init__(self, n_classes):
    super(Classifier, self).__init__()
    self.bert = RobertaModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Classifier(3)
model = model.to(device)

In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train()

  losses = []
  correct_predictions = 0
  train_batch = len(data_loader)
  for batch_index, d in enumerate(data_loader):
    if batch_index % 200 == 0:
      print(f'Batch Index: {batch_index} / {train_batch}')
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0
  valid_batch = len(data_loader)
  with torch.no_grad():
    for index, d in enumerate(data_loader):
      if index % 200 == 0:
        print(f"Batch {index} / {valid_batch}")
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
EPOCHS = 2

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()
  
  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      texts = d["review_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      #targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      #real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  #real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs

In [ ]:
# del test
# del train

In [ ]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_valid)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Epoch 1/2
----------
Batch Index: 0 / 12657
Batch Index: 200 / 12657
Batch Index: 400 / 12657
Batch Index: 600 / 12657
Batch Index: 800 / 12657
Batch Index: 1000 / 12657
Batch Index: 1200 / 12657
Batch Index: 1400 / 12657
Batch Index: 1600 / 12657
Batch Index: 1800 / 12657
Batch Index: 2000 / 12657
Batch Index: 2200 / 12657
Batch Index: 2400 / 12657
Batch Index: 2600 / 12657
Batch Index: 2800 / 12657
Batch Index: 3000 / 12657
Batch Index: 3200 / 12657
Batch Index: 3400 / 12657
Batch Index: 3600 / 12657
Batch Index: 3800 / 12657
Batch Index: 4000 / 12657
Batch Index: 4200 / 12657
Batch Index: 4400 / 12657
Batch Index: 4600 / 12657
Batch Index: 4800 / 12657
Batch Index: 5000 / 12657
Batch Index: 5200 / 12657
Batch Index: 5400 / 12657
Batch Index: 5600 / 12657
Batch Index: 5800 / 12657
Batch Index: 6000 / 12657
Batch Index: 6200 / 12657
Batch Index: 6400 / 12657
Batch Index: 6600 / 12657
Batch Index: 6800 / 12657
Batch Index: 7000 / 12657
Batch Index: 7200 / 12657
Batch Index: 7400 / 1265

In [ ]:
print(len(test_data_loader))
y_review_texts, y_pred, y_predprobs = get_predictions(
  model,
  test_data_loader
)

938


In [ ]:
preds = y_pred.tolist()
preds[0:5]

[2, 0, 0, 0, 1]

In [ ]:
np.unique(preds)

array([0, 1, 2])

In [ ]:
submission = pd.DataFrame(preds, columns=['label'])
submission.to_csv('roberta_base_epoch2.csv', index=False)